<a href="https://colab.research.google.com/github/sunghoByun/TensorflowDeveloperCertificate/blob/main/CNN_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN 실습

앞서 살펴본 예제를 바탕으로, 실제로 코드를 작성하는 실습을 진행합니다.

In [29]:
import urllib.request
import zipfile
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator

## Rock-Paper-Scissors 데이터셋 로드

Rock-Paper-Scissors 데이터셋을 로드합니다.

Rock-Paper-Scissors 데이터셋은 가위, 바위, 보 게임을 하는 손 이미지를 모은 데이터셋으로, 300x300x3 크기의 이미지들로 구성되어 있습니다.

In [30]:
url = 'https://storage.googleapis.com/download.tensorflow.org/data/rps.zip'
urllib.request.urlretrieve(url, 'rps.zip')
local_zip = 'rps.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('tmp/')
zip_ref.close()

## 로드한 데이터 확인

예시로 몇 개의 데이터만 확인해보겠습니다.

현재 경로의 tmp/rps 폴더 안에 paper, rock, scissors 폴더와 이미지가 생성된 것을 확인할 수 있습니다.

Rock-Scissors-Paper Dataset의 세부적인 정보는 아래 링크에서 확인할 수 있습니다.

https://www.tensorflow.org/datasets/catalog/rock_paper_scissors

In [31]:
!ls tmp/rps/* | head -20

tmp/rps/paper:
paper01-000.png
paper01-001.png
paper01-002.png
paper01-003.png
paper01-004.png
paper01-005.png
paper01-006.png
paper01-007.png
paper01-008.png
paper01-009.png
paper01-010.png
paper01-011.png
paper01-012.png
paper01-013.png
paper01-014.png
paper01-015.png
paper01-016.png
paper01-017.png
paper01-018.png


## **MISSION: 데이터 전처리**

##**※ 실제 시험 문제가 이와 같은 형태로 출제됩니다.**

이 데이터로 모델을 학습시키기 위해선 각종 전처리가 필요합니다.

TFDS 패키지를 이용한 것이 아니므로 ImageDataGenerator를 이용하여 전처리를 진행해야 합니다.

##**작성할 코드**

1. ImageDataGenerator를 이용하여 픽셀값 normalize를 진행하며 학습데이터를 구성하십시오.

2. 구성된 학습 데이터에 flow_from_directory를 이용하여 데이터 전처리 및 batch 사이즈를 지정하십시오.

3. (선택) Train/Validation Split을 진행하십시오.

4. (선택) ImageDataGenerator 과정에서 Augmentation 과정을 추가하십시오.

3번과 4번의 경우에는 (선택)이므로 진행하지 않아도 문제 없습니다.

<br>


ImageDataGenerator와 flow_from_directory에 관련한 상세한 자료는 아래 TensorFlow Document에서 확인하실 수 있습니다.

https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator#flow_from_directory



In [32]:
TRAINING_DIR = "tmp/rps/"

training_datagen = ImageDataGenerator(
    # TODO: 픽셀값 normalize와 학습데이터 구성
    rescale = 1./255,
    validation_split = 0.2
)

# TODO: 학습 데이터에 flow_from_directory를 이용하여 데이터 전처리 및 batch 사이즈 지정

In [33]:
#Augmentation 적용

training_datagen = ImageDataGenerator(
    rescale = 1./255,
    validation_split=0.2,
    rotation_range=5,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)
 

In [34]:
training_data = training_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size = (256,256),
    batch_size = 10,
    class_mode = 'sparse',
    subset = 'training'
)

Found 2016 images belonging to 3 classes.


In [45]:
test_data = training_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size = (256,256),
    batch_size = 10,
    class_mode = 'sparse',
    subset = 'validation'
)

Found 504 images belonging to 3 classes.


## **MISSION: 네트워크 정의**

##**※ 실제 시험 문제가 이와 같은 형태로 출제됩니다.**

위에서 변환한 데이터로 학습할 네트워크를 정의합니다.

##**작성할 코드**

Image Classification에 잘 동작하는 네트워크를 설계해보세요. 마지막 레이어는 그대로 둔 채, 앞 부분의 레이어들을 추가하시면 됩니다.

Validation Dataset이 있다면, Validation Accuracy가 80% 이상이 되도록 설계하시면 됩니다.

<br>

Conv1D, MaxPooling2D, Dense 레이어를 중심으로 구현하면 좋습니다.

In [36]:
model = tf.keras.models.Sequential([
    # TODO: Add Layers
    tf.keras.layers.Conv2D(16,(2,2),activation='relu', input_shape = (256,256,3)),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(32,(2,2),activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(64,(2,2),activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(rate = 0.3),
    tf.keras.layers.Dense(3, activation= 'softmax')
])

In [37]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 255, 255, 16)      208       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 127, 127, 16)      0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 126, 126, 32)      2080      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 62, 62, 64)        8256      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 31, 31, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 61504)            

## **MISSION: 네트워크 학습**

##**※ 실제 시험 문제가 이와 같은 형태로 출제됩니다.**

위에서 설계한 네트워크 구조를 바탕으로 compile 및 학습을 진행합니다.

##**작성할 코드**

설계한 네트워크에 적절한 Loss, Optimizer, Metrics을 지정하여 모델을 Compile 및 학습을 실시합니다.

<br>

Validation Dataset이 있다면, Validation accuracy가 80% 이상 나오도록 위의 네트워크 구조 및 학습 epoch 수를 변경하시면 좋습니다.


In [42]:
# TODO: Compile and Training
model.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss = tf.keras.losses.sparse_categorical_crossentropy,
    metrics = ['sparse_categorical_accuracy','accuracy']
)


In [39]:
model.fit(training_data, epochs= 20, validation_data= test_data)

Epoch 1/20
202/202 [==============================] - 39s 190ms/step - loss: 1.0546 - sparse_categorical_accuracy: 0.4097 - accuracy: 0.4097 - val_loss: 0.8142 - val_sparse_categorical_accuracy: 0.5556 - val_accuracy: 0.5556
Epoch 2/20
202/202 [==============================] - 38s 189ms/step - loss: 0.4330 - sparse_categorical_accuracy: 0.8104 - accuracy: 0.8104 - val_loss: 0.7000 - val_sparse_categorical_accuracy: 0.6806 - val_accuracy: 0.6806
Epoch 3/20
202/202 [==============================] - 38s 189ms/step - loss: 0.2553 - sparse_categorical_accuracy: 0.9070 - accuracy: 0.9070 - val_loss: 1.2217 - val_sparse_categorical_accuracy: 0.6250 - val_accuracy: 0.6250
Epoch 4/20
202/202 [==============================] - 38s 188ms/step - loss: 0.1832 - sparse_categorical_accuracy: 0.9334 - accuracy: 0.9334 - val_loss: 0.5276 - val_sparse_categorical_accuracy: 0.8254 - val_accuracy: 0.8254
Epoch 5/20
202/202 [==============================] - 38s 190ms/step - loss: 0.1573 - sparse_categor

In [46]:
model.evaluate(test_data)

51/51 [==============================] - 7s 145ms/step - loss: 0.6770 - sparse_categorical_accuracy: 0.8115 - accuracy: 0.8115


[0.6769545674324036, 0.8115079402923584, 0.8115079402923584]